# COEN 140: Homework 2, LDA & QDA

## Robbie Culkn

In [1]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [2]:
table = pd.read_csv('data/iris.csv',header=None)
print table.head()
table[4].value_counts()

     0    1    2    3            4
0  5.1  3.5  1.4  0.2  Iris-setosa
1  4.9  3.0  1.4  0.2  Iris-setosa
2  4.7  3.2  1.3  0.2  Iris-setosa
3  4.6  3.1  1.5  0.2  Iris-setosa
4  5.0  3.6  1.4  0.2  Iris-setosa


Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: 4, dtype: int64

# Train - Test split
#### 1. Break the sample into 80% for training, and 20% for test datasets. You can choose the first 80% instances from each class for training and the rest for testing.

In [3]:
train = table[0:40].append(table[50:90]).append(table[100:140])
test  = table[40:50].append(table[90:100]).append(table[140:150])
print train.shape
print test.shape

(120, 5)
(30, 5)


In [4]:
setosa =     train[train[4] == 'Iris-setosa']
versicolor = train[train[4] == 'Iris-versicolor']
virginica =  train[train[4] == 'Iris-virginica']

In [5]:
print 'setosa'
print setosa.head()
print '\nversicolor'
print versicolor.head()
print '\nvirginica'
print virginica.head()

setosa
     0    1    2    3            4
0  5.1  3.5  1.4  0.2  Iris-setosa
1  4.9  3.0  1.4  0.2  Iris-setosa
2  4.7  3.2  1.3  0.2  Iris-setosa
3  4.6  3.1  1.5  0.2  Iris-setosa
4  5.0  3.6  1.4  0.2  Iris-setosa

versicolor
      0    1    2    3                4
50  7.0  3.2  4.7  1.4  Iris-versicolor
51  6.4  3.2  4.5  1.5  Iris-versicolor
52  6.9  3.1  4.9  1.5  Iris-versicolor
53  5.5  2.3  4.0  1.3  Iris-versicolor
54  6.5  2.8  4.6  1.5  Iris-versicolor

virginica
       0    1    2    3               4
100  6.3  3.3  6.0  2.5  Iris-virginica
101  5.8  2.7  5.1  1.9  Iris-virginica
102  7.1  3.0  5.9  2.1  Iris-virginica
103  6.3  2.9  5.6  1.8  Iris-virginica
104  6.5  3.0  5.8  2.2  Iris-virginica


In [6]:
#drop labels
trn_setosa = setosa.drop(4,axis=1)
trn_versicolor = versicolor.drop(4,axis=1)
trn_virginica = virginica.drop(4,axis=1)

## Define Probability Density Function
see eqn 11.1 at this link: http://www.hep.caltech.edu/~NarskyPorter/Excerpts/Pages221_227.pdf

In [7]:
def P(x,mean,cov):
    n_features = x.shape[0]

    first = 1/math.sqrt(((2.0*math.pi)**n_features) * np.linalg.det(cov)) 
    second =  math.exp(-0.5*np.dot(np.dot((x-mean),np.linalg.inv(cov)),(x-mean)[np.newaxis].T) )
    
    return first*second

## Training: Compute sample mean & sample covariance

In [8]:
def my_mean(X):
    means = []
    for col in X.values.T:
        means.append(col.sum()/float(X.shape[0]))
    return np.array(means)

In [9]:
def my_cov(X):
    total_cov = np.zeros((X.shape[1],X.shape[1]))
    mean = my_mean(X)
    for row in range(X.shape[0]):
        total_cov += np.outer((X.iloc[row].values - mean),(X.iloc[row].values - mean))
    cov = total_cov/float(X.shape[0]-1) #n-1 in order to be an unbiased estimator
    return cov

In [10]:
means = {}
means['setosa']  = my_mean(trn_setosa)
means['versicolor'] = my_mean(trn_versicolor)
means['virginica'] = my_mean(trn_virginica)

covs = {}
covs['setosa'] = my_cov(trn_setosa)
covs['versicolor'] = my_cov(trn_versicolor)
covs['virginica'] = my_cov(trn_virginica)

cov_avg = (covs['setosa'] + covs['versicolor'] + covs['virginica'])/3.0

print means
print covs

{'setosa': array([ 5.0375,  3.44  ,  1.4625,  0.2325]), 'versicolor': array([ 6.01  ,  2.78  ,  4.3175,  1.35  ]), 'virginica': array([ 6.6225,  2.96  ,  5.6075,  1.99  ])}
{'setosa': array([[ 0.13112179,  0.09897436,  0.01298077,  0.01362179],
       [ 0.09897436,  0.13271795,  0.00205128,  0.0145641 ],
       [ 0.01298077,  0.00205128,  0.02958333,  0.00458333],
       [ 0.01362179,  0.0145641 ,  0.00458333,  0.00994231]]), 'versicolor': array([[ 0.27374359,  0.08661538,  0.17212821,  0.05230769],
       [ 0.08661538,  0.11087179,  0.08087179,  0.04538462],
       [ 0.17212821,  0.08087179,  0.20353205,  0.07371795],
       [ 0.05230769,  0.04538462,  0.07371795,  0.04307692]]), 'virginica': array([[ 0.46794231,  0.11041026,  0.35777564,  0.05125641],
       [ 0.11041026,  0.11323077,  0.08107692,  0.04625641],
       [ 0.35777564,  0.08107692,  0.34532692,  0.05930769],
       [ 0.05125641,  0.04625641,  0.05930769,  0.07425641]])}


# Linear Discriminant Analysis
#### 2. Build an LDA classifier based on the training data. Report the training and test errors for your classifier. 
note: using avg of 3 cov matricies as this was taught in class, but should be using a pooled cov matrix

In [11]:
def LDA(x, mu, avg_cov):
    prob = {}
    prob['Iris-setosa'] = P(x,mu['setosa'],avg_cov)
    prob['Iris-versicolor'] = P(x,mu['versicolor'],avg_cov)
    prob['Iris-virginica'] = P(x,mu['virginica'],avg_cov)
    
    return max(prob, key=prob.get)

In [12]:
LDA(trn_versicolor.iloc[0].as_matrix(),means,cov_avg)

'Iris-versicolor'

## LDA: Evaluate on Training set

In [13]:
n_correct = 0
for row in train.iterrows():
    x = np.array(row[1][0:4])
    y = row[1][4]
    if LDA(x,means,cov_avg) == y:
        n_correct += 1
accuracy = n_correct/float(len(train))
print 'Training acc:',accuracy

Training acc: 0.975


## LDA: Evaluate on Validation set

In [14]:
n_correct = 0
for row in test.iterrows():
    x = np.array(row[1][0:4])
    y = row[1][4]
    if LDA(x,means,cov_avg) == y:
        n_correct += 1
accuracy = n_correct/float(len(test))
print 'Testing acc:',accuracy

Testing acc: 1.0


# Quadratic Discriminant Analysis
#### 3. Build a QDA classifier based on the training data. Report the training and test errors for your classifier.

In [15]:
def QDA(x,means,covs):
    prob = {}
    prob['Iris-setosa'] = P(x,means['setosa'],covs['setosa'])
    prob['Iris-versicolor'] = P(x,means['versicolor'],covs['versicolor'])
    prob['Iris-virginica'] = P(x,means['virginica'],covs['virginica'])
    
    return max(prob, key=prob.get)

## QDA: Evaluate on Training set

In [16]:
n_correct = 0
for row in train.iterrows():
    x = np.array(row[1][0:4])
    y = row[1][4]
    if QDA(x,means,covs) == y:
        n_correct += 1
accuracy = n_correct/float(len(train))
print 'Training acc:',accuracy

Training acc: 0.983333333333


## QDA: Evaluate on Validation set

In [17]:
n_correct = 0
for row in test.iterrows():
    x = np.array(row[1][0:4])
    y = row[1][4]
    y_hat = QDA(x,means,covs)
    if y_hat == y:
        n_correct += 1
accuracy = n_correct/float(len(test))
print 'Testing acc:',accuracy

Testing acc: 1.0


# Linear Seperability
#### 4. Is there any class linearly separable from other classes? Explain your answer based on your experiments.

In [18]:
#do training & testing sets all at once, doesn't matter for this
categories = ['Iris-setosa','Iris-versicolor','Iris-virginica']
for category in categories:
    X = table[table[4] == category]
    n_correct = 0
    for row in X.iterrows():
        x = np.array(row[1][0:4])
        y = row[1][4]
        y_hat = LDA(x,means,cov_avg)
        if y_hat == y:
            n_correct += 1
    accuracy = n_correct/float(len(X))
    print category,'accuracy:',accuracy

Iris-setosa accuracy: 1.0
Iris-versicolor accuracy: 0.96
Iris-virginica accuracy: 0.98


From this result we may assume that the setosa class is linearly seperable, because this is the only class that our LDA classifier was able to perfectly classify.

# Feature importance
#### 5. Are any of the variables not important in classifying iris type? Explain your answer based on your experiments.
drop each feature and do QDA & LDA

In [19]:
LDA_accuracy = {}
QDA_accuracy = {}

for feature in range(trn_setosa.shape[1]):
    #drop the chosen feature
    setosa_ = trn_setosa.drop(feature,axis=1)
    versicolor_ = trn_versicolor.drop(feature,axis=1)
    virginica_ = trn_virginica.drop(feature,axis=1)
    
    test_ = test.drop(feature,axis=1)
    
    #train
    means_ = {}
    means_['setosa']  = my_mean(setosa_)
    means_['versicolor'] = my_mean(versicolor_)
    means_['virginica'] = my_mean(virginica_)

    covs_ = {}
    covs_['setosa'] = my_cov(setosa_)
    covs_['versicolor'] = my_cov(versicolor_)
    covs_['virginica'] = my_cov(virginica_)

    cov_avg_ = (covs_['setosa'] + covs_['versicolor'] + covs_['virginica'])/3.0
    
    #LDA
    n_correct = 0
    for row in test_.iterrows():
        x = np.array(row[1][0:3])
        y = row[1][4]
        if LDA(x,means_,cov_avg_) == y:
            n_correct += 1
    LDA_accuracy[feature]  = n_correct/float(len(test))
    
    #QDA  
    n_correct = 0
    for row in test_.iterrows():
        x = np.array(row[1][0:3])
        y = row[1][4]
        if QDA(x,means_,covs_) == y:
            n_correct += 1
    QDA_accuracy[feature] = n_correct/float(len(test))

In [20]:
print 'LDA:',LDA_accuracy
print 'QDA:',QDA_accuracy

LDA: {0: 1.0, 1: 1.0, 2: 1.0, 3: 0.9666666666666667}
QDA: {0: 1.0, 1: 1.0, 2: 1.0, 3: 0.9666666666666667}


From the above result we can conclude that feature 3 (Petal width?) is perhaps more important, as only its removal had an impact on testing accuracy.

# Assuming Feature Independence
#### 6. Assume the features are independent, i.e., ∑ is a diagonal matrix. Repeat 2 and 3, and report your results.

In [21]:
#convert cov matrices to diagonal (set non-diag entries to 0)
indep_covs = {}
for category,cov in covs.iteritems():
    indep_covs[category] = zeros(cov.shape)
    for row in range(cov.shape[0]):
        for col in range(cov.shape[1]):
            if row == col:
                indep_covs[category][row][col] = cov[row][col]
    print indep_covs[category]
indep_cov_avg = (indep_covs['setosa']+indep_covs['virginica']+indep_covs['versicolor'])/3.0

[[ 0.13112179  0.          0.          0.        ]
 [ 0.          0.13271795  0.          0.        ]
 [ 0.          0.          0.02958333  0.        ]
 [ 0.          0.          0.          0.00994231]]
[[ 0.27374359  0.          0.          0.        ]
 [ 0.          0.11087179  0.          0.        ]
 [ 0.          0.          0.20353205  0.        ]
 [ 0.          0.          0.          0.04307692]]
[[ 0.46794231  0.          0.          0.        ]
 [ 0.          0.11323077  0.          0.        ]
 [ 0.          0.          0.34532692  0.        ]
 [ 0.          0.          0.          0.07425641]]


## LDA: Evaluate on Training set

In [22]:
n_correct = 0
for row in train.iterrows():
    x = np.array(row[1][0:4])
    y = row[1][4]
    if LDA(x,means,indep_cov_avg) == y:
        n_correct += 1
accuracy = n_correct/float(len(train))
print 'Training acc:',accuracy

Training acc: 0.95


## LDA: Evaluate on Validation set

In [23]:
n_correct = 0
for row in test.iterrows():
    x = np.array(row[1][0:4])
    y = row[1][4]
    if LDA(x,means,indep_cov_avg) == y:
        n_correct += 1
accuracy = n_correct/float(len(test))
print 'Testing acc:',accuracy

Testing acc: 1.0


## QDA: Evaluate on Training set

In [24]:
n_correct = 0
for row in train.iterrows():
    x = np.array(row[1][0:4])
    y = row[1][4]
    if QDA(x,means,indep_covs) == y:
        n_correct += 1
accuracy = n_correct/float(len(train))
print 'Training acc:',accuracy

Training acc: 0.958333333333


## QDA: Evaluate on Validation set

In [25]:
n_correct = 0
for row in test.iterrows():
    x = np.array(row[1][0:4])
    y = row[1][4]
    y_hat = QDA(x,means,indep_covs)
    if y_hat == y:
        n_correct += 1
accuracy = n_correct/float(len(test))
print 'Testing acc:',accuracy

Testing acc: 1.0


While our testing acurracies remain perfect, training accuracies are lower than our QDA & LDA experiments without this assumption.